In [ ]:
!pip install pycuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 45.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.1 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2024.1.2-cp310-cp310-linux_x86_64.whl size=660545 sha256=70ea712c4616eb37d18c9a9f36ee0a4f7e261082970030a262d57dcf10f1d1bd
  Stored in directory: /root/.cache/pip/wheels/70/63/40/4bf006182f942d3516b71bb2ff3b57ccbdb8b2c0ee81882b6e
Successfully built pycuda


In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [1]:
%%writefile btree.cu

#include <cuda_runtime.h>
#include <iostream>
#include <vector>

#define MAX_KEYS 4  // Simplificación del factor de ramificación
#define NUM_INSERTS 10 // Número de claves a insertar en paralelo
#define NUM_SEARCHES 5 // Número de búsquedas en paralelo

// Estructura del Nodo en el B-Tree con soporte de versiones
struct BTreeNode {
    int keys[MAX_KEYS];           // Claves en el nodo
    int numKeys;                  // Número actual de claves
    BTreeNode *children[MAX_KEYS + 1];  // Punteros a hijos
    BTreeNode *nextVersion;       // Puntero a la versión anterior del nodo
    int version;                  // Versión del nodo

    // Constructor
    __host__ __device__ BTreeNode(int v = 0) : numKeys(0), version(v), nextVersion(nullptr) {
        for (int i = 0; i < MAX_KEYS + 1; i++) children[i] = nullptr;
    }
};

// Kernel para insertar múltiples claves en paralelo
__global__ void parallelInsert(BTreeNode *root, int *keys, int version) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < NUM_INSERTS) {
        // Cada hilo inserta una clave
        if (root->numKeys < MAX_KEYS) {
            root->keys[root->numKeys++] = keys[idx];
            root->version = version;
        } else {
            printf("Nodo completo. Manejo de división no implementado.\n");
        }
    }
}

// Búsqueda de clave en una versión específica del árbol
__device__ BTreeNode* searchKey(BTreeNode *root, int key, int version) {
    BTreeNode *current = root;
    while (current != nullptr) {
        if (current->version <= version) {
            for (int i = 0; i < current->numKeys; i++) {
                if (current->keys[i] == key) return current;
            }
        }
        current = current->nextVersion;
    }
    return nullptr; // Clave no encontrada en esta versión
}

// Kernel para ejecutar búsquedas en paralelo
__global__ void parallelSearch(BTreeNode *root, int *keys, int version, BTreeNode **results) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < NUM_SEARCHES) {
        results[idx] = searchKey(root, keys[idx], version);
    }
}

// Kernel para tomar snapshots (crear nuevas versiones del nodo si es necesario)
__global__ void takeSnapshot(BTreeNode *root, int newVersion) {
    if (root->version < newVersion) {
        BTreeNode *newNode = new BTreeNode(newVersion);
        for (int i = 0; i < root->numKeys; i++) {
            newNode->keys[i] = root->keys[i];
        }
        newNode->numKeys = root->numKeys;
        newNode->nextVersion = root;
        *root = *newNode; // Actualizar a la nueva versión
    }
}

// Función principal
int main() {
    BTreeNode *root;
    cudaMallocManaged(&root, sizeof(BTreeNode));  // Memoria unificada
    *root = BTreeNode(0);  // Versión inicial

    // Asignar claves de prueba
    int h_keys[NUM_INSERTS] = {10, 20, 30, 40, 50, 60, 70, 80, 90, 100};
    int *d_keys;
    cudaMalloc(&d_keys, NUM_INSERTS * sizeof(int));
    cudaMemcpy(d_keys, h_keys, NUM_INSERTS * sizeof(int), cudaMemcpyHostToDevice);

    // Inserciones paralelas
    parallelInsert<<<1, NUM_INSERTS>>>(root, d_keys, 1);
    cudaDeviceSynchronize();

    // Tomar snapshot
    takeSnapshot<<<1, 1>>>(root, 2);
    cudaDeviceSynchronize();

    // Búsquedas paralelas
    int h_search_keys[NUM_SEARCHES] = {10, 50, 100, 60, 90};
    int *d_search_keys;
    cudaMalloc(&d_search_keys, NUM_SEARCHES * sizeof(int));
    cudaMemcpy(d_search_keys, h_search_keys, NUM_SEARCHES * sizeof(int), cudaMemcpyHostToDevice);

    BTreeNode **d_results;
    cudaMalloc(&d_results, NUM_SEARCHES * sizeof(BTreeNode*));
    parallelSearch<<<1, NUM_SEARCHES>>>(root, d_search_keys, 2, d_results);
    cudaDeviceSynchronize();

    // Copiar resultados a la CPU y mostrar
    BTreeNode *h_results[NUM_SEARCHES];
    cudaMemcpy(h_results, d_results, NUM_SEARCHES * sizeof(BTreeNode*), cudaMemcpyDeviceToHost);
    for (int i = 0; i < NUM_SEARCHES; i++) {
        if (h_results[i] != nullptr) {
            printf("Clave %d encontrada en la versión 2.\n", h_search_keys[i]);
        } else {
            printf("Clave %d no encontrada en la versión 2.\n", h_search_keys[i]);
        }
    }

    // Liberar memoria
    cudaFree(root);
    cudaFree(d_keys);
    cudaFree(d_search_keys);
    cudaFree(d_results);
    return 0;
}


Writing btree.cu


In [2]:
!nvcc -o btree btree.cu

btree.cu:5:3: error: invalid preprocessing directive #Definimos
    5 | # Definimos la estructura del nodo del B-Tree
      |   ^~~~~~~~~
btree.cu:39:3: error: invalid preprocessing directive #Funci\U000000f3n
   39 | # Función para insertar claves en paralelo
      |   ^~~~~~~
btree.cu:45:3: error: invalid preprocessing directive #Funci\U000000f3n
   45 | # Función para buscar claves en paralelo
      |   ^~~~~~~
btree.cu:53:3: error: invalid preprocessing directive #Funci\U000000f3n
   53 | # Función para eliminar claves en paralelo
      |   ^~~~~~~
btree.cu:62:3: error: invalid preprocessing directive #Funci\U000000f3n
   62 | # Función para tomar snapshot (crear nuevas versiones del nodo)
      |   ^~~~~~~
btree.cu:73:3: error: invalid preprocessing directive #Funci\U000000f3n
   73 | # Función principal para simular el comportamiento
      |   ^~~~~~~
btree.cu:75:7: error: invalid preprocessing directive #Nodo
   75 |     # Nodo raíz del B-Tree
      |       ^~~~
btree.cu:78:7: e

In [ ]:
!./btree

^C


In [9]:
import concurrent.futures
import random

# Definimos el factor de ramificación y otros parámetros
MAX_KEYS = 4  # Máximo de claves antes de dividir
NUM_INSERTS = 10  # Número de claves a insertar en paralelo
NUM_SEARCHES = 5  # Número de búsquedas en paralelo
NUM_DELETES = 3  # Número de eliminaciones en paralelo

class BTreeNode:
    def __init__(self, version=0):
        self.keys = []  # Claves en el nodo
        self.children = [None] * (MAX_KEYS + 1)  # Punteros a hijos
        self.next_version = None  # Puntero a la versión anterior del nodo
        self.version = version  # Versión del nodo

    def insert_key(self, key):
        # Inserta la clave en orden
        if len(self.keys) < MAX_KEYS:
            self.keys.append(key)
            self.keys.sort()
        else:
            print(f"Nodo completo. Manejo de división no implementado en la versión {self.version}.")

    def search_key(self, key):
        if key in self.keys:
            return self
        return None

    def delete_key(self, key):
        if key in self.keys:
            self.keys.remove(key)
            return True
        return False

# Función para dividir un nodo lleno
def split_node(parent, node, version):
    mid_index = len(node.keys) // 2
    mid_key = node.keys[mid_index]

    # Crear el nuevo nodo que contendrá la mitad de las claves
    new_node = BTreeNode(version)
    new_node.keys = node.keys[mid_index + 1:]
    new_node.children = node.children[mid_index + 1:]

    # Actualizar el nodo original
    node.keys = node.keys[:mid_index]
    node.children = node.children[:mid_index + 1]

    # Insertar la clave central en el nodo padre
    if parent is None:
        # Crear una nueva raíz si no hay un padre
        new_root = BTreeNode(version)
        new_root.keys = [mid_key]
        new_root.children = [node, new_node]
        return new_root
    else:
        # Insertar la clave en el nodo padre y añadir el nuevo nodo como hijo
        parent.keys.append(mid_key)
        parent.keys.sort()
        parent.children.insert(parent.keys.index(mid_key) + 1, new_node)
        return parent

# Función para manejar la inserción con divisiones
def insert_key_with_split(root, key, version):
    node = root
    stack = []  # Pila para rastrear el camino hacia el nodo hoja

    # Navegar hasta el nodo hoja
    while node is not None:
        stack.append(node)
        if len(node.keys) < MAX_KEYS:
            node.insert_key(key)
            break
        elif len(node.keys) >= MAX_KEYS:
            # Si el nodo está lleno, realizamos la división
            if len(stack) > 1:
                parent = stack[-2]
                split_node(parent, node, version)
            else:
                # Si el nodo raíz está lleno, dividir y crear una nueva raíz
                root = split_node(None, node, version)
            break

    return root  # Retornar la nueva raíz si la estructura cambia

# Funciones paralelizadas para inserción, búsqueda y eliminación
def parallel_insert(root, keys, version):
    for key in keys:
        root = insert_key_with_split(root, key, version)
    root.version = version
    return root

def parallel_search(root, keys, version):
    results = []
    for key in keys:
        result = root.search_key(key)
        results.append((key, result))
    return results

def parallel_delete(root, keys, version):
    results = []
    for key in keys:
        success = root.delete_key(key)
        results.append((key, success))
    root.version = version
    return results

# Función para tomar snapshot (crear nuevas versiones del nodo)
def take_snapshot(root, new_version):
    if root.version < new_version:
        new_node = BTreeNode(new_version)
        new_node.keys = root.keys[:]
        new_node.children = root.children[:]
        new_node.next_version = root
        return new_node
    return root

# Función principal para ejecutar el ejemplo
def main():
    # Nodo raíz del B-Tree
    root = BTreeNode(0)

    # Asignar claves de prueba
    keys_to_insert = [random.randint(1, 100) for _ in range(NUM_INSERTS)]

    # Inserciones en paralelo
    with concurrent.futures.ThreadPoolExecutor() as executor:
        chunks = [keys_to_insert[i:i + 2] for i in range(0, len(keys_to_insert), 2)]
        futures = [executor.submit(parallel_insert, root, chunk, 1) for chunk in chunks]
        concurrent.futures.wait(futures)

    # Tomar snapshot después de la inserción
    root = take_snapshot(root, 2)

    # Claves para búsqueda
    search_keys = [random.randint(1, 100) for _ in range(NUM_SEARCHES)]

    # Realizar búsquedas en paralelo
    with concurrent.futures.ThreadPoolExecutor() as executor:
        chunks = [search_keys[i:i + 2] for i in range(0, len(search_keys), 2)]
        futures = [executor.submit(parallel_search, root, chunk, 2) for chunk in chunks]
        results = []
        for future in concurrent.futures.as_completed(futures):
            results.extend(future.result())

    # Mostrar resultados de búsquedas
    for key, result in results:
        if result:
            print(f"Clave {key} encontrada en la versión 2.")
        else:
            print(f"Clave {key} no encontrada en la versión 2.")

    # Claves para eliminar
    keys_to_delete = random.sample(keys_to_insert, NUM_DELETES)

    # Realizar eliminaciones en paralelo
    with concurrent.futures.ThreadPoolExecutor() as executor:
        chunks = [keys_to_delete[i:i + 2] for i in range(0, len(keys_to_delete), 2)]
        futures = [executor.submit(parallel_delete, root, chunk, 3) for chunk in chunks]
        delete_results = []
        for future in concurrent.futures.as_completed(futures):
            delete_results.extend(future.result())

    # Mostrar resultados de eliminaciones
    for key, success in delete_results:
        if success:
            print(f"Clave {key} eliminada en la versión 3.")
        else:
            print(f"Clave {key} no encontrada para eliminar en la versión 3.")

if __name__ == "__main__":
    main()


Clave 52 no encontrada en la versión 2.
Clave 52 no encontrada en la versión 2.
Clave 60 no encontrada en la versión 2.
Clave 4 no encontrada en la versión 2.
Clave 83 no encontrada en la versión 2.
Clave 77 no encontrada para eliminar en la versión 3.
Clave 60 no encontrada para eliminar en la versión 3.
Clave 98 no encontrada para eliminar en la versión 3.
